## Quadratic Programming 

In [ ]:
import numpy as np
from numpy.linalg import solve, lstsq, norm, LinAlgError
import time
import pandas as pd
from typing import List, Tuple, Dict

In [25]:
def project_onto_l1_ball(v: np.ndarray, radius: float = 1.0) -> np.ndarray:
    """Project *v* onto {x ∈ ℝⁿ | ‖x‖₁ ≤ radius}.  Works in *O(n log n)*.

    Based on the algorithm of *Duchi et al. (2008)*.
    """
    if radius < 0:
        raise ValueError("radius must be non negative")

    abs_v = np.abs(v)
    if abs_v.sum() <= radius:
        return v.copy()           # already feasible

    # Sort |v| in descending order
    u = np.sort(abs_v)[::-1]
    cssv = np.cumsum(u)
    rho = np.nonzero(u * np.arange(1, len(u) + 1) > cssv - radius)[0][-1]
    theta = (cssv[rho] - radius) / (rho + 1)

    w = np.sign(v) * np.maximum(abs_v - theta, 0.0)
    return w


def true_solution_qp(x_hat: np.ndarray) -> tuple[np.ndarray, float]:
    """Return the analytic minimiser of the project QP.

    The objective is ½‖x-x̂‖² subject to ‖x‖₁ ≤ 1.
    This is simply the *Euclidean projection* of *x_hat* onto
    the L1-ball of radius 1.  The optimal value is ½‖x*-x̂‖².
    """
    x_star = project_onto_l1_ball(x_hat, radius=1.0)
    f_star = 0.5 * norm(x_star - x_hat) ** 2
    return x_star, f_star



In [ ]:
class ActiveSetQP:
    """Primal active-set solver for the L₁-ball projection QP.

    Parameters
    ----------
    x_hat : np.ndarray
        Vector we project onto the L₁-ball.
    eps : float, optional (default=1e-10)
        Ridge added to the *z*-block of the Hessian to guarantee a
        nonsingular KKT system when the working set is empty or
        momentarily rank-deficient.
    max_iter : int, optional (default = 20 times n)
        Hard iteration cap.
    tol : float, optional (default = 1e-8)
        Tolerance for primal feasibility and Lagrange-multiplier tests.
    """

    def __init__(self, x_hat: np.ndarray, *, eps: float = 1e-10,
                 max_iter: int | None = None, tol: float = 1e-8) -> None:
        self.x_hat = np.asarray(x_hat, dtype=float)
        self.n     = self.x_hat.size
        self.eps   = float(eps)
        self.max_iter = max_iter or (20 * self.n)   # ← raise cap
        self.tol      = tol

        # Build (G, c, A, b) once – they never change ---------------
        self.G, self.c, self.A, self.b = self._build_qp_matrices()
        self.m = self.A.shape[0]                      # number of constraints
        
    def solve(self, x0: np.ndarray | None = None) -> Tuple[np.ndarray, float, Dict]:
        """Run the primal active‑set loop and return (x*, f*, info)."""
        # ------------------------------------------------------------------
        # 0. Start from a **feasible** point y₀ = [x₀; z₀].  If caller did
        #    not give one, use the true projection – that guarantees
        #    feasibility and gives the shortest path to convergence.
        # ------------------------------------------------------------------
        if x0 is None:
            x  = project_onto_l1_ball(self.x_hat)            # feasible
        else:
            x  = np.asarray(x0, dtype=float)
            # If infeasible -> project to get feas.
            if np.abs(x).sum() > 1.0 + self.tol:
                x = project_onto_l1_ball(x)

        z = np.abs(x)                         # minimal z that supports x
        if z.sum() > 1.0 + self.tol:
            # scale down uniformly to satisfy Σ zᵢ ≤ 1
            z *= 1.0 / z.sum()
        y = np.concatenate([x, z])            # working primal vector

        # Initial working set – include Σ z ≤ 1 at least --------------
        W = self._initial_working_set(y)
        if not W:          # guarantee KKT has full rank in first step
            W = [3 * self.n]     # index of Σ z ≤ 1 constraint

        # Helper lambdas ---------------------------------------------
        grad = lambda yy: self.G @ yy + self.c
        obj  = lambda yy: 0.5 * yy @ (self.G @ yy) + self.c @ yy

        # ------------------------------------------------------------------
        # Iterate until optimal or until we hit `max_iter`.
        for k in range(self.max_iter):
            g   = grad(y)
            # 1. Solve the equality‑constrained sub‑QP (KKT system) ---
            p = self._solve_equality_qp(y, W, g)
            if np.linalg.norm(p) <= self.tol:
                # (a) Search direction is 0 ⇒ check Lagrange multipliers
                lambda_hat = self._compute_multipliers(y, W, g)
                if not lambda_hat.size or lambda_hat.min() >= -self.tol:
                    # KKT conditions satisfied – done
                    info = {
                        "status":   "optimal",
                        "iter":     k,
                        "n_active": len(W),
                    }
                    x_star = y[:self.n]
                    f_star = obj(y)
                    return x_star, f_star, info

                # Otherwise drop the most negative multiplier --------
                idx_to_drop = W[int(lambda_hat.argmin())]
                W.remove(idx_to_drop)
                continue                        # and repeat outer loop

            # (b) Search direction non‑zero – compute max α that keeps
            #     all constraints satisfied.
            alpha, blocking_idx = self._step_length(y, p)
            y = y + alpha * p

            if blocking_idx is not None:        # new constraint hits
                W.append(blocking_idx)
                # ensure unique indices
                W = list(dict.fromkeys(W))
        # loop finished without convergence
        info = {
            "status":   "max_iter_reached",
            "iter":     self.max_iter,
            "n_active": len(W),
        }
        return y[:self.n], obj(y), info

    # Helper methods --------------------------------------------------
    def _build_qp_matrices(self):
        n = self.n
        # Hessian G (block‑diagonal) – ridge ε on z‑block ------------
        G = np.diag(np.concatenate([np.ones(n), self.eps * np.ones(n)]))
        c = np.concatenate([-self.x_hat, np.zeros(n)])

        # Build constraint matrix A and RHS b  -----------------------
        I_n  = np.eye(n)

        rows = [
            np.hstack([ I_n, -I_n]),      # x - z ≤ 0
            np.hstack([-I_n, -I_n]),      # -x - z ≤ 0
            np.hstack([np.zeros((1, n)), np.ones((1, n))]),  # Σ z ≤ 1
            np.hstack([np.zeros((n, n)), -I_n]),             # -z ≤ 0
        ]
        rhs  = [
            np.zeros(n),
            np.zeros(n),
            np.array([1.0]),
            np.zeros(n),
        ]

        A = np.vstack(rows)
        b = np.concatenate(rhs)
        return G, c, A, b

    # ------------------------------------------------------------------
    def _initial_working_set(self, y: np.ndarray) -> List[int]:
        residuals = self.A @ y - self.b
        return np.where(np.abs(residuals) <= self.tol)[0].tolist()

    # ------------------------------------------------------------------
    def _solve_equality_qp(self, y: np.ndarray, W: List[int], g: np.ndarray) -> np.ndarray:
        """Return the minimising *p* of the sub-QP with *A_W p = 0*."""
        if not W:                             # no active constraints
            try:
                return -solve(self.G, g)      # plain Newton step
            except LinAlgError:
                p, *_ = lstsq(self.G, -g, rcond=None)
                return p

        A_W = self.A[W]
        KKT  = np.block([
            [self.G,  A_W.T],
            [A_W,     np.zeros((len(W), len(W)))],
        ])
        rhs = -np.concatenate([g, np.zeros(len(W))])
        try:
            sol = solve(KKT, rhs)
        except LinAlgError:
            delta = self.eps
            KKT_reg = KKT + delta * np.eye(KKT.shape[0])
            sol = solve(KKT_reg, rhs)
        return sol[: self.G.shape[0]]

    # ------------------------------------------------------------------
    def _compute_multipliers(self, y: np.ndarray, W: List[int], g: np.ndarray) -> np.ndarray:
        if not W:
            return np.array([])
        A_W = self.A[W]
        KKT = np.block([
            [self.G,  A_W.T],
            [A_W,     np.zeros((len(W), len(W)))],
        ])
        rhs = -np.concatenate([g, np.zeros(len(W))])
        try:
            sol = solve(KKT, rhs)
        except LinAlgError:
            sol, *_ = lstsq(KKT, rhs, rcond=None)
        return sol[self.G.shape[0]:]

    # ------------------------------------------------------------------
    def _step_length(self, y: np.ndarray, p: np.ndarray) -> Tuple[float, int | None]:
        """Compute max a ∈ (0,1] with A(y+ap) ≤ b.

        Returns
        -------
        alpha : float
            Feasible step size.
        blocking_idx : int | None
            Index of constraint that becomes active (if any).
        """
        Ay = self.A @ y
        Ap = self.A @ p

        # Tightening constraints satisfy Ap > +tol 
        mask = Ap > self.tol
        if not mask.any():
            return 1.0, None

        alphas = (self.b[mask] - Ay[mask]) / Ap[mask]
        # Numerical safety: ignore very small negative due to round‑off
        alphas = np.maximum(alphas, 0.0)
        alpha  = alphas.min()

        # First index achieving alpha (tie‑break deterministic)
        idxs   = np.where(mask)[0]
        blocking_idx = int(idxs[alphas.argmin()])

        # Clamp alpha to [0,1]
        alpha = min(max(alpha, 0.0), 1.0)
        return alpha, blocking_idx


In [ ]:
def run_all_tests_custom(seed: int = 0) -> pd.DataFrame:
    """
    Run the LP simplex solver *and* the QP active-set solver on the
    ‖x‖₁ ≤ 1 test-suite.

    Returns
    -------
    pd.DataFrame
        One row per (dimension, start-vector) pair with detailed
        timing / success / quality metrics for both solvers.
    """
    rng = np.random.default_rng(seed)     
    dims = [2, 3, 4, 5, 10, 20, 30, 50, 100, 200]
    starts = lambda n: [np.zeros(n), np.ones(n)]
    results = []

    for n in dims:
        for x0 in starts(n):

            vec = rng.standard_normal(n)

            # QP via the new active-set class
            t0 = time.time()
            try:
                solver = ActiveSetQP(x_hat=vec,max_iter=10000)
                x_qp, f_qp, info = solver.solve(x0=x0)
                qp_success = (info.get("status") == "optimal")
                qp_err = "" if qp_success else info.get("status", "solver-returned-no-status")
            except Exception as err:
                x_qp = f_qp = None
                qp_success, qp_err = False, str(err)
            qp_time = time.time() - t0

            
            results.append({
                "n": n,
                "start": "zeros" if not np.any(x0) else "ones",
                "QP_val":       f_qp,
                "QP_success":   qp_success,
                "QP_error":     qp_err,
                "QP_norm_l1":   None if x_qp is None else np.linalg.norm(x_qp, 1),
                "QP_norm_l2":   None if x_qp is None else np.linalg.norm(x_qp, 2),
                "QP_time_sec":  qp_time,
            })

    return pd.DataFrame(results)


In [23]:
df = run_all_tests_custom()
print(df)

      n  start    QP_val  QP_success          QP_error  QP_norm_l1  \
0     2  zeros -0.012678        True                      0.132105   
1     2   ones -0.210573        True                      0.745323   
2     3  zeros -0.817418        True                      1.000000   
3     3   ones -0.883962        True                      1.000000   
4     4  zeros -1.825031        True                      1.000000   
5     4   ones -0.658493        True                      0.735984   
6     5  zeros -0.980724        True                      1.000000   
7     5   ones -0.623336        True                      1.000000   
8    10  zeros -0.624418        True                      1.000000   
9    10   ones -1.565681        True                      1.000000   
10   20  zeros -1.545958        True                      1.000000   
11   20   ones -1.826878        True                      1.000000   
12   30  zeros -1.801059        True                      1.000000   
13   30   ones -1.88